# 네이버 증권 개별종목 일자별 과거 시세

In [1]:
import requests 
import pandas as pd
from bs4 import BeautifulSoup as bs
import sqlite3
import re

* request의 get()를 사용하여 읽어보겠습니다.

In [2]:
requests.get('https://finance.naver.com/item/sise_day.nhn?code=005930&page=4').text

'\n\n\n\n\n\n\n<html lang="ko">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr">\n<title>네이버 금융</title>\n<link rel="stylesheet" type="text/css" href="/css/newstock.css?20190627113311">\n<link rel="stylesheet" type="text/css" href="/css/common.css?20190627113311">\n<link rel="stylesheet" type="text/css" href="/css/layout.css?20190627113311">\n<link rel="stylesheet" type="text/css" href="/css/main.css?20190627113311">\n<link rel="stylesheet" type="text/css" href="/css/newstock2.css?20190627113311">\n<link rel="stylesheet" type="text/css" href="/css/newstock3.css?20190627113311">\n<link rel="stylesheet" type="text/css" href="/css/world.css?20190627113311">\n</head>\n<body>\n<script language="JavaScript">\nfunction mouseOver(obj){\n  obj.style.backgroundColor="#f6f4e5";\n}\nfunction mouseOut(obj){\n  obj.style.backgroundColor="#ffffff";\n}\n</script>\n\t\t\t\t<h4 class="tlline2"><strong><span class="red03">일별</span>시세</strong></h4>\t\t\t\n\t\t\t\t<table cellspa

### read_html() 메소드 사용하기
* 예전에 배운 pandas의 read_html() 메소드를 사용하여 table 형태로 되어 있는 웹 페이지를 쉽게 DataFrame 형태로 가져올 수 있습니다.
* 아래 주소의 값으로 데이터를 가져오면 2개의 데이터를 가져옵니다. 
* 첫번째 데이터는 웹 페이지에서 보이는 날짜, 종가, 전일비, 시가, 고가, 저가, 거래량 정보가 있는 데이터이며, 
* 두번째 데이터는 페이지 정보입니다. 
* **첫번째 정보만 사용하겠습니다.**

In [3]:
len(pd.read_html("https://finance.naver.com/item/sise_day.nhn?code=005930&page=2"))

2

In [4]:
pd.read_html("https://finance.naver.com/item/sise_day.nhn?code=005930&page=2")[0]

,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019.07.05,45650.0,350.0,45950.0,45950.0,45250.0,7235395.0
2,2019.07.04,46000.0,600.0,45250.0,46200.0,45250.0,6365573.0
3,2019.07.03,45400.0,850.0,45750.0,46350.0,45200.0,9669368.0
4,2019.07.02,46250.0,350.0,46200.0,46900.0,45850.0,8463073.0
5,2019.07.01,46600.0,400.0,47350.0,47400.0,46250.0,11383522.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2019.06.28,47000.0,500.0,47000.0,47000.0,46700.0,12949231.0


In [5]:
pd.read_html("https://finance.naver.com/item/sise_day.nhn?code=005930&page=2")[1]

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,맨앞,1,2,3,4,5,6,7,8,9,10,다음,맨뒤


* NaN은 결측치로 사용할 수 없는 값입니다. 
* DataFrame에서는 이 값을 dropna() 메소드를 사용하면 쉽게 NaN이 포함된 행을 제거할 수 있습니다.

In [6]:
data = pd.read_html("https://finance.naver.com/item/sise_day.nhn?code=005930&page=2")[0].dropna()

In [7]:
data

,날짜,종가,전일비,시가,고가,저가,거래량
1,2019.07.05,45650.0,350.0,45950.0,45950.0,45250.0,7235395.0
2,2019.07.04,46000.0,600.0,45250.0,46200.0,45250.0,6365573.0
3,2019.07.03,45400.0,850.0,45750.0,46350.0,45200.0,9669368.0
4,2019.07.02,46250.0,350.0,46200.0,46900.0,45850.0,8463073.0
5,2019.07.01,46600.0,400.0,47350.0,47400.0,46250.0,11383522.0
9,2019.06.28,47000.0,500.0,47000.0,47000.0,46700.0,12949231.0
10,2019.06.27,46500.0,800.0,46000.0,46600.0,45750.0,12603534.0
11,2019.06.26,45700.0,100.0,45800.0,46000.0,45600.0,9226097.0
12,2019.06.25,45600.0,100.0,45200.0,45800.0,45200.0,7076774.0
13,2019.06.24,45500.0,200.0,45200.0,45800.0,45200.0,6085066.0


## sqliite3를 사용합니다.
* stock.db에 저장되어 있는 master 테이블의 값을 사용하겠습니다.

In [2]:
con = sqlite3.connect("./common_stock.db")

In [3]:
cur = con.cursor()

* master 테이블에 종목의 코드명이 들어있습니다.

In [4]:
master = pd.read_sql("select * from master", con=con)

In [5]:
master.head(3)

,codeName,full_code,marketName,short_code
0,가비아,KR7079940003,KOSDAQ,079940
1,가온미디어,KR7078890001,KOSDAQ,078890
2,가온전선,KR7000500009,KOSPI,000500


### sqlite의 5가지 데이터 타입
* NULL	: NULL values mean missing information or unknown.
* INTEGER :	Integer values are whole numbers (either positive or negative). An integer can have variable sizes such as 1, 2,3, 4, or 8 bytes.
* REAL	: Real values are real numbers with decimal values that use 8-byte floats.
* TEXT	: TEXT is used to store character data. The maximum length of TEXT is unlimited. SQLite supports various character encodings.
* BLOB	: BLOB stands for a binary large object that can be used to store any kind of data. The maximum size of BLOBs is unlimited.

#### table를 직접 작성해보겠습니다.

In [21]:
query = """CREATE TABLE "stock_day_table" (
    "종목이름" TEXT NOT NULL,
    "종목코드" TEXT NOT NULL,
    "날짜" TEXT NOT NULL,
    "종가" REAL,
    "전일비" REAL,
    "시가" REAL,
    "고가" REAL,
    "저가" REAL,
    "거래량" REAL,
    PRIMARY KEY('종목이름', '종목코드', "날짜")
)"""

In [22]:
con.execute(query)

In [14]:
data['종목코드'] = '005930'
data['종목이름'] = '삼성전자'

In [15]:
data

,날짜,종가,전일비,시가,고가,저가,거래량,종목코드,종목이름
1,2019.07.05,45650.0,350.0,45950.0,45950.0,45250.0,7235395.0,005930,삼성전자
2,2019.07.04,46000.0,600.0,45250.0,46200.0,45250.0,6365573.0,005930,삼성전자
3,2019.07.03,45400.0,850.0,45750.0,46350.0,45200.0,9669368.0,005930,삼성전자
4,2019.07.02,46250.0,350.0,46200.0,46900.0,45850.0,8463073.0,005930,삼성전자
5,2019.07.01,46600.0,400.0,47350.0,47400.0,46250.0,11383522.0,005930,삼성전자
9,2019.06.28,47000.0,500.0,47000.0,47000.0,46700.0,12949231.0,005930,삼성전자
10,2019.06.27,46500.0,800.0,46000.0,46600.0,45750.0,12603534.0,005930,삼성전자
11,2019.06.26,45700.0,100.0,45800.0,46000.0,45600.0,9226097.0,005930,삼성전자
12,2019.06.25,45600.0,100.0,45200.0,45800.0,45200.0,7076774.0,005930,삼성전자
13,2019.06.24,45500.0,200.0,45200.0,45800.0,45200.0,6085066.0,005930,삼성전자


In [16]:
data.to_sql("stock_table_day", con=con, if_exists='append', index=False)

In [23]:
cursor = con.execute("Select * from stock_table_day")

In [24]:
cursor.fetchall()

[('2019.07.05',
  45650.0,
  350.0,
  45950.0,
  45950.0,
  45250.0,
  7235395.0,
  '005930',
  '삼성전자'),
 ('2019.07.04',
  46000.0,
  600.0,
  45250.0,
  46200.0,
  45250.0,
  6365573.0,
  '005930',
  '삼성전자'),
 ('2019.07.03',
  45400.0,
  850.0,
  45750.0,
  46350.0,
  45200.0,
  9669368.0,
  '005930',
  '삼성전자'),
 ('2019.07.02',
  46250.0,
  350.0,
  46200.0,
  46900.0,
  45850.0,
  8463073.0,
  '005930',
  '삼성전자'),
 ('2019.07.01',
  46600.0,
  400.0,
  47350.0,
  47400.0,
  46250.0,
  11383522.0,
  '005930',
  '삼성전자'),
 ('2019.06.28',
  47000.0,
  500.0,
  47000.0,
  47000.0,
  46700.0,
  12949231.0,
  '005930',
  '삼성전자'),
 ('2019.06.27',
  46500.0,
  800.0,
  46000.0,
  46600.0,
  45750.0,
  12603534.0,
  '005930',
  '삼성전자'),
 ('2019.06.26',
  45700.0,
  100.0,
  45800.0,
  46000.0,
  45600.0,
  9226097.0,
  '005930',
  '삼성전자'),
 ('2019.06.25',
  45600.0,
  100.0,
  45200.0,
  45800.0,
  45200.0,
  7076774.0,
  '005930',
  '삼성전자'),
 ('2019.06.24',
  45500.0,
  200.0,
  45200.0,
  458

* 마지막 페이지(pgRR)

In [25]:
rt = bs(requests.get("https://finance.naver.com/item/sise_day.nhn?code=005930&page=2").text, 'html.parser')

In [26]:
rt


<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>네이버 금융</title>
<link href="/css/newstock.css?20190627113311" rel="stylesheet" type="text/css"/>
<link href="/css/common.css?20190627113311" rel="stylesheet" type="text/css"/>
<link href="/css/layout.css?20190627113311" rel="stylesheet" type="text/css"/>
<link href="/css/main.css?20190627113311" rel="stylesheet" type="text/css"/>
<link href="/css/newstock2.css?20190627113311" rel="stylesheet" type="text/css"/>
<link href="/css/newstock3.css?20190627113311" rel="stylesheet" type="text/css"/>
<link href="/css/world.css?20190627113311" rel="stylesheet" type="text/css"/>
</head>
<body>
<script language="JavaScript">
function mouseOver(obj){
  obj.style.backgroundColor="#f6f4e5";
}
function mouseOut(obj){
  obj.style.backgroundColor="#ffffff";
}
</script>
<h4 class="tlline2"><strong><span class="red03">일별</span>시세</strong></h4>
<table cellspacing="0" class="type2">
<tr>
<th>날짜</th>
<th>종가</th

In [27]:
rt.find("td", class_='pgRR').find("a")['href']

'/item/sise_day.nhn?code=005930&page=581'

In [28]:
pattern = re.compile("(\d+)")

In [29]:
pattern.findall(rt.find("td", class_='pgRR').find("a")['href'])[-1]

'581'

## 코드를 정리하여 2400개여의 종목의 과거 시세를 저장하겠습니다. 

In [30]:
master.loc[0]

codeName               가비아
full_code     KR7079940003
marketName          KOSDAQ
short_code          079940
Name: 0, dtype: object

In [72]:
# loc() 메소드를 사용하면 row 인덱스 값으로 데이터를 추출할 수 있습니다.
a, b, c, d = master.loc[0]

In [73]:
a

'가비아'

In [74]:
b

'KR7079940003'

In [75]:
c

'KOSDAQ'

In [76]:
d

'A079940'

In [35]:
for cont in master.iterrows():
    pass
    break

In [37]:
cont[1]

codeName               가비아
full_code     KR7079940003
marketName          KOSDAQ
short_code          079940
Name: 0, dtype: object

* 종목의 마지막 페이지를 찾아보겠습니다.

In [33]:
url =  "https://finance.naver.com/item/sise_day.nhn?code={}&page={}"
url.format(123013, 1)

'https://finance.naver.com/item/sise_day.nhn?code=123013&page=1'

In [16]:
bs(requests.get(url.format(code, 1)).text, 'html.parser').find("td", class_='pgRR')

* 이제 for문을 돌려서 db에 입력합니다.
* 테스트로 첫번째 페이지만 입력합니다.

In [100]:
url =  "https://finance.naver.com/item/sise_day.nhn?code={}&page={}"
pattern = re.compile("(\d+)")
for stock in master.iterrows():
    company, _1, _2, code = stock[1]
    print (company)
    # 새롭게 생긴 종목은 1페이지만 존재할수 있다. 
    try:
        last = pattern.findall(bs(requests.get(url.format(code, 1)).text, 'html.parser').find("td", class_='pgRR').find("a")['href'])[-1]
    except:
        last = 1
    for cnt in range(1, int(last)+1):
    #for cnt in range(1, 2):
        #print (cnt)
        data = pd.read_html(url.format(code, cnt))[0].dropna()
        data['종목이름'] = company
        data['종목코드'] = code
        # 데이터가 중복되어 있다면 primary key 값으로 인하여 오류가 발생한다.
        try:
            data.to_sql('stock_day_table', con=con, if_exists='append', index=False)
        except:
            pass

가비아
가온미디어
가온전선
감마누
강남제비스코
강스템바이오텍
강원랜드
갤럭시아에스엠
갤럭시아컴즈
게임빌
경남스틸
경남제약
경농
경동나비엔
경동도시가스
경동인베스트
경동제약
경방
경보제약
경인양행
경인전자
경창산업
계룡건설
계양전기
계양전기우
고려개발
고려산업
고려시멘트
고려신용정보
고려아연
고려제강
고려제약
고영
골드퍼시픽
골든센츄리
골프존
골프존뉴딘홀딩스
골프존데카
관악산업
광동제약
광림
광명전기
광전자
광주신세계
광진실업
광진윈텍
교보7호스팩
교보8호스팩
교보증권
구영테크
국도화학
국동
국보
국보디자인
국순당
국영지앤엠
국일신동
국일제지
국제약품
굿센
그린케미칼
그린플러스
극동유화
글로벌에스엠
글로벌텍스프리
글로본
금강공업
금강공업우
금강철강
금비
금양
금호산업
금호산업우
금호석유
금호석유우
금호에이치티
금호전기
금호타이어
금화피에스시
기가레인
기산텔레콤
기신정기
기아차
기업은행
까뮤이앤씨
까스텔바쟉
깨끗한나라
깨끗한나라우
나노
나노메딕스
나노스
나노신소재
나노엔텍
나노캠텍
나눔테크
나라소프트
나라엠앤디
나무가
나무기술
나스미디어
나우아이비캐피탈
나이벡
나이스디앤비
나이스정보통신
남광토건
남선알미늄
남선알미우
남성
남양유업
남양유업우
남영비비안
남해화학
남화산업
남화토건
내츄럴엔도텍
네오디안테크놀로지
네오셈
네오오토
네오위즈
네오위즈홀딩스
네오티스
네오팜
네오펙트
네이블
네이처셀
네추럴FNP
네패스
네패스신소재
넥센
넥센우
넥센타이어
넥센타이어1우B
넥스턴
넥스트BT
넥스트사이언스
넥스트아이
넥슨지티
넵튠
넷게임즈
넷마블
노랑풍선
노루페인트
노루페인트우
노루홀딩스
노루홀딩스우
노바렉스
노바텍
노브메타파마
녹십자
녹십자랩셀
녹십자셀
녹십자엠에스
녹십자홀딩스
녹십자홀딩스2우
녹원씨엔아이
농심
농심홀딩스
농우바이오
누리텔레콤
누리플랜
뉴로스
뉴보텍
뉴인텍
뉴지랩
뉴트리
뉴파워프라즈마
뉴프라이드
뉴프렉스
다나와
다날
다믈멀티미디어
다산네트웍스
다스코
다우기술
다우데이타
다원넥스뷰
다원시스
다이노나
다이오진
대교
대교우B
대구백화점
대덕전자
대덕전자1우
대동고려삼


인프라웨어
인프라웨어테크놀러지
인피니트헬스케어
인화정공
일동제약
일동홀딩스
일성건설
일성신약
일신바이오
일신방직
일신석재
일야
일양약품
일양약품우
일정실업
일지테크
일진다이아
일진디스플
일진머티리얼즈
일진전기
일진파워
일진홀딩스
잇츠한불
잉글우드랩
잉크테크
자비스
자연과환경
자이글
자화전자
장원테크
재영솔루텍
전방
전우정밀
전진바이오팜
전파기지국
정다운
정산애강
정상제이엘에스
정원엔시스
제낙스
제너셈
제넥신
제넨바이오
제노레이
제노텍
제노포커스
제놀루션
제닉
제로투세븐
제룡산업
제룡전기
제미니투자
제우스
제이브이엠
제이스테판
제이스텍
제이씨케미칼
제이씨현시스템
제이에스코퍼레이션
제이에스티나
제이엔케이히터
제이엠아이
제이엠티
제이웨이
제이준코스메틱
제이콘텐트리
제이테크놀로지
제이티
제일기획
제일바이오
제일약품
제일연마
제일제강
제일테크노스
제일파마홀딩스
제주반도체
제주은행
제주항공
젠큐릭스
젬백스
조광ILI
조광페인트
조광피혁
조비
조선내화
조선선재
조아제약
조이맥스
조이시티
조일알미늄
조흥
종근당
종근당바이오
종근당홀딩스
좋은사람들
주노콜렉션
주성엔지니어링
주연테크
줌인터넷
중앙백신
중앙에너비스
중앙오션
지노믹트리
지놈앤컴퍼니
지니뮤직
지니언스
지란지교시큐리티
지스마트글로벌
지앤이헬스케어
지어소프트
지에스이
지엔씨에너지
지엔코
지엘팜텍
지엠비코리아
지엠피
지역난방공사
지오씨
지와이커머스
지코
지투알
지투하이소닉
지트리비앤티
지티지웰니스
진도
진로발효
진매트릭스
진바이오텍
진성티이씨
진양산업
진양제약
진양폴리
진양홀딩스
진양화학
진에어
진원생명과학
진흥기업
진흥기업2우B
진흥기업우B
질경이
차바이오텍
차이나그레이트
참엔지니어링
참좋은여행
창해에탄올
천보
천일고속
청광종건
청담러닝
청보산업
청호컴넷
체리부로
체시스
초록뱀
칩스앤미디어
카스
카이노스메드
카카오
카페24
카프로
캐스텍코리아
캔서롭
캠시스
컨버즈
컬러레이
컴투스
컴퍼니케이
케미메디
케어랩스
케어젠
케이디켐
케이맥
케이비17호스팩
케이비아이동국실업
케이비제10호스팩
케이비제11호스팩
케이비제18호스팩
케

* 입력된 데이터를 확인해보겠습니다.

In [101]:
cursor = con.execute("Select * from stock_day_table")

In [102]:
cursor.fetchall()

[('2019.07.16',
  2410.0,
  15.0,
  2420.0,
  2445.0,
  2390.0,
  26736.0,
  'iMBC',
  '052220'),
 ('2019.07.15',
  2425.0,
  30.0,
  2450.0,
  2480.0,
  2415.0,
  27716.0,
  'iMBC',
  '052220'),
 ('2019.07.12',
  2455.0,
  5.0,
  2455.0,
  2490.0,
  2425.0,
  35786.0,
  'iMBC',
  '052220'),
 ('2019.07.11',
  2450.0,
  15.0,
  2440.0,
  2485.0,
  2425.0,
  34332.0,
  'iMBC',
  '052220'),
 ('2019.07.10',
  2435.0,
  10.0,
  2400.0,
  2450.0,
  2390.0,
  48177.0,
  'iMBC',
  '052220'),
 ('2019.07.09',
  2425.0,
  55.0,
  2475.0,
  2550.0,
  2425.0,
  106149.0,
  'iMBC',
  '052220'),
 ('2019.07.08',
  2480.0,
  90.0,
  2620.0,
  2620.0,
  2450.0,
  71525.0,
  'iMBC',
  '052220'),
 ('2019.07.05',
  2570.0,
  40.0,
  2535.0,
  2615.0,
  2530.0,
  78939.0,
  'iMBC',
  '052220'),
 ('2019.07.04',
  2530.0,
  25.0,
  2590.0,
  2590.0,
  2530.0,
  83762.0,
  'iMBC',
  '052220'),
 ('2019.07.03',
  2555.0,
  15.0,
  2565.0,
  2625.0,
  2550.0,
  99879.0,
  'iMBC',
  '052220'),
 ('2019.07.17',
  82

In [103]:
pd.read_sql_query("select * from stock_day_table", con=con)

,날짜,종가,전일비,시가,고가,저가,거래량,종목이름,종목코드
0,2019.07.16,2410.0,15.0,2420.0,2445.0,2390.0,26736.0,iMBC,052220
1,2019.07.15,2425.0,30.0,2450.0,2480.0,2415.0,27716.0,iMBC,052220
2,2019.07.12,2455.0,5.0,2455.0,2490.0,2425.0,35786.0,iMBC,052220
3,2019.07.11,2450.0,15.0,2440.0,2485.0,2425.0,34332.0,iMBC,052220
4,2019.07.10,2435.0,10.0,2400.0,2450.0,2390.0,48177.0,iMBC,052220
5,2019.07.09,2425.0,55.0,2475.0,2550.0,2425.0,106149.0,iMBC,052220
6,2019.07.08,2480.0,90.0,2620.0,2620.0,2450.0,71525.0,iMBC,052220
7,2019.07.05,2570.0,40.0,2535.0,2615.0,2530.0,78939.0,iMBC,052220
8,2019.07.04,2530.0,25.0,2590.0,2590.0,2530.0,83762.0,iMBC,052220
9,2019.07.03,2555.0,15.0,2565.0,2625.0,2550.0,99879.0,iMBC,052220


## 정규식을 사용하여 마지막 페이지 가져오기(추가)

In [56]:
test_r = requests.get("https://finance.naver.com/item/sise_day.nhn?code=005930&page=1")

In [57]:
test_bs = bs(test_r.text, "html.parser")

In [58]:
test_bs


<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>네이버 금융</title>
<link href="/css/newstock.css?20190627113311" rel="stylesheet" type="text/css"/>
<link href="/css/common.css?20190627113311" rel="stylesheet" type="text/css"/>
<link href="/css/layout.css?20190627113311" rel="stylesheet" type="text/css"/>
<link href="/css/main.css?20190627113311" rel="stylesheet" type="text/css"/>
<link href="/css/newstock2.css?20190627113311" rel="stylesheet" type="text/css"/>
<link href="/css/newstock3.css?20190627113311" rel="stylesheet" type="text/css"/>
<link href="/css/world.css?20190627113311" rel="stylesheet" type="text/css"/>
</head>
<body>
<script language="JavaScript">
function mouseOver(obj){
  obj.style.backgroundColor="#f6f4e5";
}
function mouseOut(obj){
  obj.style.backgroundColor="#ffffff";
}
</script>
<h4 class="tlline2"><strong><span class="red03">일별</span>시세</strong></h4>
<table cellspacing="0" class="type2">
<tr>
<th>날짜</th>
<th>종가</th

In [59]:
test_pattern = re.compile("/item/sise_day.nhn\?code=[0-9A-Z]+&amp;page=([0-9]+)")

In [60]:
test_pattern.findall(test_r.text)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '581']

In [61]:
url =  "https://finance.naver.com/item/sise_day.nhn?code={}&page={}"
pattern = re.compile("/item/sise_day.nhn\?code=[0-9A-Z]+&amp;page=([0-9]+)")
for stock in master.iterrows():
    company, _1, _2, code = stock[1]
    print (company)
    # 새롭게 생긴 종목은 1페이지만 존재할수 있다. 
    try:
        last = pattern.findall(requests.get(url.format(code, 1)).text)[-1]
    except:
        print ("마지막 페이지 에러")
        print (company)
        break 
        
    print (last)
    
    for cnt in range(1, int(last)+1):
    #for cnt in range(1, 2):
        data = pd.read_html(url.format(code, cnt))[0].dropna()
        data['종목이름'] = company
        data['종목코드'] = code
        # 데이터가 중복되어 있다면 primary key 값으로 인하여 오류가 발생한다.
        try:
            data.to_sql('stock_day_table', con=con, if_exists='append', index=False)
        except Exception as e :
            print (e)

가비아
341
